In [1]:
##########################################
# General helper functions and libraries #
##########################################

# General data libs
import pandas as pd
import numpy as np

# General plotting libs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import animation
import plotly.express as px
import seaborn as sns

####################
# Modeling process #
####################

# Networks
import networkx as nx
from pyvis.network import Network
from community import community_louvain

In [2]:
# Based on: https://www.bis.org/statistics/r_qa1406_hanx9b.pdf
# Originally from "Consolidated Banking Statistics, Updated 27 July 2023" described at: https://www.bis.org/statistics/consstats.htm
# Full set here: https://www.bis.org/statistics/full_data_sets.htm
# Retrieved on 13 Oct

# File, will take ~15sec to load
df = pd.DataFrame(pd.read_csv('https://storage.googleapis.com/suptech-lab-practical-data-science-public/WS_CBS_PUB_csv_col.csv'))
df.shape

(174575, 151)

In [3]:
df.columns

Index(['FREQ', 'Frequency', 'L_MEASURE', 'Measure', 'L_REP_CTY',
       'Reporting country', 'CBS_BANK_TYPE', 'CBS bank type', 'CBS_BASIS',
       'CBS reporting basis',
       ...
       '2020-Q4', '2021-Q1', '2021-Q2', '2021-Q3', '2021-Q4', '2022-Q1',
       '2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1'],
      dtype='object', length=151)

In [ ]:
# Immediate borrower basis
set(df['CBS reporting basis'])